In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

In [6]:
filename = '../Gestion allocations'
o_stock_df = pd.read_excel(f'{filename}.xlsx', sheet_name=1, skiprows=0, thousands=',')
o_orders_df = pd.read_excel(f'{filename}.xlsx', sheet_name=0, skiprows=0)

In [8]:
o_stock_df[o_stock_df['Article'] == '028059']

,Article,Stock corrigé
462,028059,3412386.116


In [115]:
if o_stock_df.iloc[-1]['Étiquettes de lignes'] == 'Total général':
    print('Removing row ', o_stock_df.tail(1), ' from stock')
    o_stock_df.drop(o_stock_df.tail(1).index, inplace=True)

if o_orders_df.iloc[-1]['Étiquettes de lignes'] == 'Total général':
    print('Removing row ', o_orders_df.tail(1), ' from orders')
    o_orders_df.drop(o_orders_df.tail(1).index, inplace=True)

Removing row        Étiquettes de lignes N° commande CN  ANMOIS Article  Somme de RAL
16965        Total général            NaN     NaN     NaN        345643  from orders


In [4]:
SRC_ORDERS_ID = 'N° commande CN'
SRC_ORDERS_ARTICLE = 'Article'
SRC_ORDERS_QTY = 'Somme de RAL'

SRC_STOCK_ID = 'Article'
SRC_STOCK_QTY = 'Qté US'

orders_df = o_orders_df[[SRC_ORDERS_ID, SRC_ORDERS_ARTICLE, SRC_ORDERS_QTY]].rename({
    SRC_ORDERS_ID: 'order_id',
    SRC_ORDERS_ARTICLE: 'article_id',
    SRC_ORDERS_QTY: 'quantity'
}, axis=1)

stock_df = o_stock_df.reset_index().rename({
    SRC_STOCK_ID: 'article_id',
    SRC_STOCK_QTY: 'stock'
}, axis=1).groupby('article_id').sum(numeric_only=True).reset_index()

# some stocks are negative, fix this
stock_df.loc[stock_df['stock'] < 0, 'stock'] = 0

tmp_order_df = pd.merge(orders_df, stock_df, on='article_id', how='left')
missing_stock_ids = tmp_order_df[tmp_order_df.stock.isna()].article_id.unique()
missing_stock_df = pd.DataFrame({'article_id': missing_stock_ids, 'stock': np.zeros_like(missing_stock_ids)})

print(f"# stock item type missing = {missing_stock_df.shape[0]}")
print(f"# orders affected = {tmp_order_df[tmp_order_df.stock.isna()].order_id.nunique()}")

stock_df = pd.concat([stock_df, missing_stock_df]).reset_index(drop=True)
stock_df['article_index'] = stock_df.index

orders_df = pd.merge(orders_df, stock_df, on='article_id')

order_id_to_idx = dict([ (oid, idx) for (idx, oid) in enumerate(orders_df['order_id'].unique()) ])
orders_df['order_index'] = orders_df['order_id'].map(order_id_to_idx)

orders_df['stock_available'] = orders_df['quantity'] <= orders_df['stock']
order_to_available_df = orders_df.groupby('order_id')['stock_available'].all()
orders_df =  pd.merge(orders_df, order_to_available_df, on='order_id', suffixes=('', '_order_total'))

o_orders_df.shape[0], orders_df.shape[0]

# stock item type missing = 3025
# orders affected = 1805


(20052, 20052)

In [5]:
print(f"# orders = {orders_df['order_id'].nunique()}")
print(f"# order that cannot be fulfilled = {order_to_available_df.shape[0] - order_to_available_df.sum()}")
print(f"# order that can be fulfilled and will be optimzied over = {order_to_available_df.sum()}")
print(f"\ttotal quantity = {orders_df[orders_df['stock_available_order_total']].quantity.sum()}")

# orders = 4341
# order that cannot be fulfilled = 2001
# order that can be fulfilled and will be optimzied over = 2340
	total quantity = 74162


In [6]:
def make_order_matrix(orders_df, num_orders, num_items):
    order_qty_matrix = np.zeros((num_items, num_orders))
    for _, order in orders_df.iterrows():
        order_qty_matrix[order['article_index'], order['order_index']] += order['quantity']
    return order_qty_matrix

num_items = stock_df['article_index'].max()+1
num_orders = orders_df['order_index'].max()+1

order_qty_matrix = make_order_matrix(orders_df, num_orders, num_items)
order_selection = cp.Variable(name='order_selection', shape=(num_orders,), boolean=True)
order_vol = orders_df.groupby('order_index')['quantity'].sum().values

print(f"orders x stock matrix = {order_qty_matrix.shape}")
print(f"# orders in books = {order_selection.shape}")

prob = cp.Problem(cp.Maximize( order_vol.T @ order_selection ),
                 [order_qty_matrix @ order_selection <= stock_df['stock']])

print("optimizing...")
prob.solve()
order_selection_sol = np.round(order_selection.value)

# check if rouned solution is still good
_o = order_qty_matrix @ order_selection_sol

print("found solution")
print('\t Solution valid =', not (_o - stock_df['stock'].values > 1e-10).any())

should_deliver = orders_df['order_index'].map(dict(enumerate(order_selection_sol)))

print(f"# orders fulfilled = {order_selection_sol.sum()}")
print(f"quantity delivered = {orders_df[should_deliver.astype(bool)].quantity.sum()}")
print(f"\t Opt = {(order_qty_matrix @ order_selection_sol).sum()}")

print(f'saving to {filename}_result.xlsx')
orders_df['should_deliver'] = orders_df['order_index'].map(dict(enumerate(order_selection_sol)))
o_df = orders_df[['order_id', 'article_id', 'quantity', 'should_deliver']]
o_df.to_excel(f'{filename}_result.xlsx')

o_df[should_deliver.astype(bool)]

orders x stock matrix = (28503, 4341)
# orders in books = (4341,)
optimizing...
found solution
	 Solution valid = True
# orders fulfilled = 1421.0
quantity delivered = 65983
	 Opt = 65983.0
saving to ../Test 2 mai b_result.xlsx


,order_id,article_id,quantity,should_deliver
16,CNS1220210600314,033488T0064S0344,8,1.0
22,CNS1220210900003,039662T0044S0344,1,1.0
23,CNS1220220402809,039662T0044S0344,1,1.0
24,CNS1220220402809,037449T0050L0086,1,1.0
25,CNS1220220403465,039662T0044S0344,2,1.0
...,...,...,...,...
20029,CNS1220220900131,53627,1,1.0
20030,CNS1220220900131,53628,1,1.0
20031,CNS1220220900131,57768,2,1.0
20032,CNS1220220900131,58555,2,1.0


In [13]:
o_df[['order_id', 'should_deliver']].groupby(['order_id', 'should_deliver']).min().reset_index()

,order_id,should_deliver
0,CNS1220201203452,0.0
1,CNS1220201205372,1.0
2,CNS1220210304774,0.0
3,CNS1220210500670,1.0
4,CNS1220210600314,1.0
...,...,...
4336,CPS1220230400022,0.0
4337,CPS1220230400023,1.0
4338,CPS1220230400024,0.0
4339,CPS1220230400025,1.0


In [20]:
orders_df[should_deliver.astype(bool)].quantity.sum()

66813

In [19]:
(order_qty_matrix @ order_selection_sol).sum()

66402.0